In [11]:
from sklearn import preprocessing
import numpy as np
import math

class NeuralNetwork:    
    def __init__(self, n_inputs, n_hidden, n_outputs, lr, activation='relu'):
        self.n_outputs = n_outputs
        self.lr = lr
        self.activation = activation
        self.layers = []
        # input layer
        self.layers.append(Layer(n_inputs, n_hidden[0], lr, activation))
        # hidden layers
        for i in range(len(n_hidden)-1):
            self.layers.append(Layer(n_hidden[i], n_hidden[i+1], lr, activation))
        # output layer
        self.layers.append(Layer(n_hidden[-1], n_outputs, lr, activation))
        
    def feed_forward(self, X):
        self.layers[0].activate(X)
        for i in range(1,len(self.layers)):
            self.layers[i].activate(self.layers[i-1].outputs)
        return self.layers[-1].outputs
    
    def back_propagate(self, X, y):
        # compute output layer gamma
        nn_output = self.feed_forward(X)
        error = nn_output - y
        gammas = error * self.activation_prime(nn_output)
        
        layer_inputs = self.layers[-2].outputs
        self.layers[-1].update_weights(layer_inputs, gammas, self.lr)
        
        #calculate hidden layer gammas
        for i in range(2, len(self.layers)+1):
            if i == len(self.layers):
                layer_inputs = X            
            else: layer_inputs = self.layers[-i-1].outputs
            layer_outputs = self.layers[-i].outputs
            weights = np.transpose(self.layers[-i+1].weights)
            gammas = np.sum(gammas*weights, axis=1) * self.activation_prime(layer_outputs)
            self.layers[-i].update_weights(layer_inputs, gammas, self.lr)   
            
    def train(self, X, y, convert=False, epochs=100, display_step=10):
        if convert:
            y = self.convert_targets(y)
        for epoch in range(epochs):
            error = 0.
            for i in range(len(X)):
                batch_X = X[i]
                batch_y = y[i]
                if self.n_outputs != len(batch_y):
                    print('Wrong dimensions of target list!')
                self.back_propagate(batch_X, batch_y)
                error += self.compute_error(batch_X, batch_y)
            
            mse = (1/len(X)) * error
            if (epoch+1) % display_step == 0:
                    print('Completed epoch', '%d' % (epoch+1), '/ %d' % epochs,
                        'Error', '%.5f' % mse)
                

    def activation_prime(self, x):
        if self.activation == 'sigmoid':
            return x*(1.0-x)
        elif self.activation == 'tanh':
            return 1.0-(x*x)
        elif self.activation == 'relu':
            return np.heaviside(x,0)
    
    def softmax(self, w, t = 1.0):
        e = np.exp(np.array(w) / t)
        dist = e / np.sum(e)
        return dist
    
    def predict(self, X):
        pred = self.feed_forward(X)
        return pred.argmax()
        
    def predict_prob(self, X):
        return self.softmax(self.feed_forward(X))
    
    def convert_targets(self, targets):        
        enc = preprocessing.LabelEncoder()
        enc.fit(targets)
        conv_targets = []
        n_classes = len(enc.classes_)
        
        conv_targets = []
        for target in targets:
            cnt = 0
            for cl in enc.classes_:
                if target == enc.classes_[cnt]:
                    arr = np.zeros(n_classes)
                    arr[cnt] = 1
                    conv_targets.append(arr)
                cnt += 1
        return conv_targets
      
    def compute_error(self, X, y):
        # X/y must be value pair for one training sample
        nn_outputs = self.feed_forward(X)
        # element wise error for each output neuron -->
        # square it --> add them --> divide by number of samples
        error = (1/len(X)) * sum(np.square(nn_outputs - y))
        return error

class Layer:
    def __init__(self, n_inputs, n_outputs, lr, activation):
        self.lr = lr
        self.activation = activation
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.weights = []
        self.bias = []
        for i in range(n_outputs):
            self.weights.append([np.random.randn() for _ in range(n_inputs)])
            self.bias.append(np.random.randn())
            
    def activation_fn(self, x):
        if self.activation == 'sigmoid':
            return 1.0/(1.0+np.exp(-x))
        elif self.activation == 'tanh':
            return np.tanh(x)
        elif self.activation == 'relu':
            return np.maximum(x,0)
                            
    def activate(self, inputs):
        outputs = []
        for i in range(self.n_outputs): 
            # multiply weights by inputs
            outputs.append(sum(np.multiply(self.weights[i], inputs)))
        # add bias and apply activation function
        self.outputs = np.add(outputs, self.bias)
        self.outputs = self.activation_fn(self.outputs)
        return outputs
    
    def update_weights(self, inputs, gammas, lr):
        for o in range(self.n_outputs):
            for i in range(len(self.weights[o])):
                self.weights[o][i] -= lr * gammas[o] * inputs[i]                
                self.bias[o] -= lr * gammas[o]

In [12]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data, labels, test_size=0.3)
train_y = np.reshape(train_y, (len(train_y),1))

scaler = preprocessing.MinMaxScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.fit_transform(test_X)

nn = NeuralNetwork(4,[2,3],3, lr=0.08, activation='sigmoid')
nn.train(train_X, train_y, convert=True, epochs=200, display_step=20)

acc = 0.
for j in range(len(test_y)):
    if test_y[j] == nn.predict(test_X[j]):
        acc += 1
print('Test accuracy:', '%.4f' % (acc/len(test_y)))

E:\Programme\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Completed epoch 20 / 200 Error 0.12686
Completed epoch 40 / 200 Error 0.08687
Completed epoch 60 / 200 Error 0.07773
Completed epoch 80 / 200 Error 0.07115
Completed epoch 100 / 200 Error 0.06561
Completed epoch 120 / 200 Error 0.06120
Completed epoch 140 / 200 Error 0.05575
Completed epoch 160 / 200 Error 0.04463
Completed epoch 180 / 200 Error 0.02991
Completed epoch 200 / 200 Error 0.01937
Test accuracy: 0.9778
